In [2]:
#used to convert the pdf to an image
from pdf2image import convert_from_path
#exact text from images using pytesseract
import pytesseract 
import cv2
import numpy as np
#Image from Pillow makes the pic printable as a image and not only an array
from PIL import Image
import re

In [3]:
pages = convert_from_path(r'docs\prescription\pre_1.pdf', poppler_path= r'C:\Users\nessa\poppler-22.04.0\Library\bin')

In [4]:
pages[0].show()


In [5]:
# first try to extract text without pre-processing anything
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang="eng")
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Maria Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

—momennannenncmneneunnmnnnnninsissiyoinnitnahaadaanih issn earnttneenrenen:

Prednisone 20 mg
Lialda 2.4 gram

3 days,

or 1 month



In [6]:
def preprocess_image(image):
    # image comes in and array comes out
    image_gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(image_gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    # we need adaptive thresholding because there are many different shade colors in the image
    processed = cv2.adaptiveThreshold(
        resized, 255,
        # specify that it is the adaptive threst
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        # and that the pic gonna be white&black
        cv2.THRESH_BINARY,
        # block size of the black area
        61,
        # constant
        11
    )
    return processed

In [7]:
# i send an image, but i get back an array so we need pillow
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [8]:
# let's try to exact the text after the pre-processing
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(img, lang="eng")
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Marta Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

K

Prednisone 20 mg
Lialda 2.4 gram

Directions:

Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks a
Lialda - take 2 pill everyday for 1 month

Refill: 2 times



In [9]:
# time to decode the text!
# starting with the name
pattern = "Name:(.+)Date:"
math = re.findall(pattern, text)
math[0].strip()

'Marta Sharapova'

In [10]:
# now comes the address
pattern = "Address:(.*)\n"
math = re.findall(pattern, text)
math[0].strip()

'9 tennis court, new Russia, DC'

In [47]:
# now comes the prescription
pattern = "Address[^\n]*(.*)Directions:"
# re.DOTALL because now we are working with multiple lines where we dont want \n to count
math = re.findall(pattern, text, flags = re.DOTALL)
math = "".join(math)
math = math.replace('K', '')


In [12]:
pattern = "Directions:(.*)Refill:"
math = re.findall(pattern, text, flags = re.DOTALL)
math = math[0].strip()
math = math.split("\n")
math

['Prednisone, Taper 5 mg every 3 days,',
 'Finish in 2.5 weeks a',
 'Lialda - take 2 pill everyday for 1 month']

In [13]:
# now comes the refill
pattern = "Refill:(.*)times"
math = re.findall(pattern, text)
math[0].strip()

'2'